In [10]:
# Import necessary libraries
import numpy as np
import cv2
import os
from scipy.spatial.distance import cdist

# Import custom functions
from methods.preprocessing import load_dataset
from methods.extract_features import extract_features_sift
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
def classify_with_sift(train_features, test_features, train_labels):
    predictions = []
    for test_feature in test_features:
        if test_feature is not None:
            # Compute distances between the test feature and all train features
            distances = np.array([cdist([test_feature], [train_feature], 'euclidean') if train_feature is not None else np.inf for train_feature in train_features])
            # Find the nearest neighbor
            nearest_index = np.argmin(distances)
            predictions.append(train_labels[nearest_index])
        else:
            # Handle cases with no SIFT features detected
            predictions.append(-1)  # Undefined or majority class
    return predictions

In [3]:
# Step 1: Load the dataset
train_dir = "../data/fer-2013-data/train"
X_train, y_train = load_dataset(train_dir)

In [4]:
# Step 2: Extract SIFT features for training data
X_train_sift = extract_features_sift(images=X_train)

In [5]:
# Step 3: Load and prepare the test dataset similarly
test_dir = "../data/fer-2013-data/test"
X_test, y_test = load_dataset(test_dir)

In [6]:
# Step 4: Extract SIFT features for testing data
X_test_sift = extract_features_sift(images=X_test)

In [7]:
# Step 5: Classify test images using the nearest neighbor approach
predictions = classify_with_sift(X_train_sift, X_test_sift, y_train)

In [8]:
# Step 6: Calculate the accuracy manually
correct_predictions = sum(1 for true, pred in zip(y_test, predictions) if true == pred and pred != -1)
total_predictions = sum(1 for pred in predictions if pred != -1)
accuracy = correct_predictions / total_predictions

In [11]:
print("SIFT Euclidian Classification Accuracy: ", accuracy)
print(classification_report(y_true=y_test, y_pred=predictions))

SIFT Euclidian Classification Accuracy:  0.3403141361256545
              precision    recall  f1-score   support

           0       0.27      0.33      0.30       958
           1       0.33      0.39      0.35      1024
           2       0.39      0.29      0.33      1774
           3       0.31      0.27      0.29      1233
           4       0.31      0.30      0.31      1247
           5       0.44      0.57      0.50       831

    accuracy                           0.34      7067
   macro avg       0.34      0.36      0.35      7067
weighted avg       0.34      0.34      0.34      7067

